# Storybook

## Install Necessary Libraries

In [1]:
!pip install transformers torch
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 775.4 kB/s eta 0:00:00 0:00:01
  Using cached regex-2024.5.15-cp310-cp310-macosx_10_9_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 6.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 3.4 MB/s eta 0:00:00:00:01
Using cached fsspec-2024.5.0-py3-none-any.whl (316 kB)
Using cached regex-2024.5.15-cp310-cp310-macosx_10_9_x86_64.whl (281 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.8/415.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00a 0:00:01
Using cached tqdm-4.6

### Import Libraries

In [2]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, GPT2Tokenizer, GPT2LMHeadModel
from PIL import Image
import requests
from io import BytesIO
import IPython.display as display

/Users/marius/.pyenv/versions/3.10.6/envs/storybook/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load BLIP Model and Processor

In [3]:
blip_model_name = "Salesforce/blip-image-captioning-large"
blip_processor = BlipProcessor.from_pretrained(blip_model_name)
blip_model = BlipForConditionalGeneration.from_pretrained(blip_model_name)

## Define Function to Generate Caption

In [4]:
def generate_caption(image_url):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    inputs = blip_processor(images=image, return_tensors="pt")
    outputs = blip_model.generate(**inputs)

    caption = blip_processor.decode(outputs[0], skip_special_tokens=True)
    return caption


### Test Image Captioning

In [8]:
# Example image URL
image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRzn-z0Nh-DBQQpd1l4fGgI9ouleVFBrSnsUw&s"
# Display image
display.display(display.Image(url=image_url))
# Generate caption
caption = generate_caption(image_url)
print("Caption:", caption)

/home/onurcanskurt/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Caption: there is a small child playing with toy cars on the floor


## Define Function to Generate Captions for Multiple Images

In [9]:
def generate_captions(image_urls):
    captions = []
    for url in image_urls:
        caption = generate_caption(url)
        captions.append(caption)
    return captions

### Example Usage for Multiple Captions

In [10]:
# Example list of image URLs
image_urls = [
    "https://cdn.cdnparenting.com/articles/2018/06/418806355-H-1024x700.jpg",
    "https://littletikescommercial.com/wp-content/uploads/2020/12/Adventureland-Park-IA_199-scaled.jpg",
    "https://www.kidstuffplaysystems.com/wp-content/uploads/2019/06/81110-Ring-trek.jpg"
]

# Generate captions
captions = generate_captions(image_urls)
for idx, caption in enumerate(captions):
    display.display(display.Image(url=image_urls[idx]))
    print(f"Caption {idx+1}:", caption)


Caption 1: several children are playing with a frisbee in a grassy field


Caption 2: there are two children playing on a playground slide


Caption 3: there are many children playing on a playground structure


In [11]:
captions

['several children are playing with a frisbee in a grassy field',
 'there are two children playing on a playground slide',
 'there are many children playing on a playground structure']

## Load GPT-2 Model and Tokenizer

In [12]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")


## Define Functions for Story Generation

In [13]:
def create_story_prompt(captions, theme):
    combined_captions = "\n".join(captions)
    prompt = (
        f"Create a captivating and coherent children's story with the following elements:\n"
        f"Captions:\n{combined_captions}\n"
        f"Theme: {theme}\n"
        f"Make sure the story is engaging and appropriate for children. "
        f"Avoid repeating sentences and ensure the story flows naturally from one scene to the next. "
        f"The story should be a fun, engaging, and imaginative adventure.\n"
        f"Story starts here: "  # Clear indicator for the model to start story generation
    )
    return prompt

def remove_repetitive_sentences(story):
    sentences = story.split('. ')
    unique_sentences = []
    seen = set()

    for sentence in sentences:
        if sentence not in seen:
            unique_sentences.append(sentence)
            seen.add(sentence)

    return '. '.join(unique_sentences)

def generate_story_gpt2(captions, theme, word_count, temperature=0.7, top_p=0.9):
    prompt = create_story_prompt(captions, theme)

    inputs = gpt2_tokenizer.encode(prompt, return_tensors="pt")
    outputs = gpt2_model.generate(
        inputs,
        max_length=word_count + len(inputs[0]),  # Account for prompt length
        num_return_sequences=1,
        temperature=temperature,
        top_p=top_p,
        no_repeat_ngram_size=2,  # Helps to reduce repetition
        pad_token_id=gpt2_tokenizer.eos_token_id  # Ensures padding uses the EOS token
    )

    story = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Remove the prompt from the generated text
    story = story[len(prompt):].strip()
    return remove_repetitive_sentences(story)

## Generate a Story

In [14]:
# Example usage
# captions = [
#     "A little boy standing on a hill.",
#     "The boy looking at a spaceship.",
#     "The spaceship taking off into the sky.",
#     "The boy inside the spaceship, smiling.",
#     "The spaceship landing on Mars."
# ]
theme = "Dragons"
word_count = 250  # Adjust based on age-appropriateness

# Generate the story
story = generate_story_gpt2(captions, theme, word_count)
print("Generated Story:", story)


/home/onurcanskurt/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/onurcanskurt/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated Story: The story begins with two kids playing in the grass. They are both playing a game of dragon. One of the kids is playing the dragon and the other is trying to catch a dragon in his mouth. Both of them are trying their best to get the ball out of his throat. As the game progresses, the two boys are getting closer and closer to catching the Dragon. When the boys catch the Dragons, they are able to escape the area and get back to their parents. This is a great way to introduce the characters to each other and to your children as well. If you are a fan of dragons, you will love this story. It is also a good way for your kids to learn about the world of Dragon Ball Z. You will also love the fact that the children play with dragons. There are several different types of Dragons. Some are simple, some are complex, others are more complex. Each type of character has their own unique abilities and abilities. For example, if you have a Dragon that is very powerful, it will be able